In [1]:
!pip install alpaca-trade-api

In [2]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
import talib

--2019-08-17 21:01:50--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 216.105.38.13
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|216.105.38.13|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2019-08-17 21:01:50--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 216.105.38.13
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: https://phoenixnap.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2019-08-17 21:01:50--  https://phoenixnap.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving phoenixnap

ModuleNotFoundError: No module named 'talib'

In [ ]:
import alpaca_trade_api as tradeapi
import time
import datetime
from datetime import timedelta
from pytz import timezone
tz = timezone('EST')

api = tradeapi.REST('YOUR_API_KEY',
                    'YOUR_SECRET')

import logging
logging.basicConfig(filename='./new_5min_ema.log', format='%(name)s - %(levelname)s - %(message)s')
logging.warning('{} logging started'.format(datetime.datetime.now().strftime("%x %X")))

def get_data_bars(symbols, rate, slow, fast):
    data = api.get_barset(symbols, rate, limit=20).df
    for x in symbols:
        data.loc[:, (x, 'fast_ema')] = data[x]['close'].rolling(window=fast).mean()
        data.loc[:, (x, 'slow_ema')] = data[x]['close'].rolling(window=slow).mean()
    return data

def get_signal_bars(symbol_list, rate, ema_slow, ema_fast):
    data = get_data_bars(symbol_list, rate, ema_slow, ema_fast)
    signals = {}
    for x in symbol_list:
        if data[x].iloc[-1]['fast_ema'] > data[x].iloc[-1]['slow_ema']: signal = 1
        else: signal = 0
        signals[x] = signal
    return signals

def time_to_open(current_time):
    if current_time.weekday() <= 4:
        d = (current_time + timedelta(days=1)).date()
    else:
        days_to_mon = 0 - current_time.weekday() + 7
        d = (current_time + timedelta(days=days_to_mon)).date()
    next_day = datetime.datetime.combine(d, datetime.time(9, 30, tzinfo=tz))
    seconds = (next_day - current_time).total_seconds()
    return seconds

def run_checker(stocklist):
    print('run_checker started')
    while True:
        # Check if Monday-Friday
        if datetime.datetime.now(tz).weekday() >= 0 and datetime.datetime.now(tz).weekday() <= 4:
            # Checks market is open
            print('Trading day')
            if datetime.datetime.now(tz).time() > datetime.time(9, 30) and datetime.datetime.now(tz).time() <= datetime.time(15, 30):
                signals = get_signal_bars(stocklist, '5Min', 20, 5)
                for signal in signals:
                    if signals[signal] == 1:
                        if signal not in [x.symbol for x in api.list_positions()]:
                            logging.warning('{} {} - {}'.format(datetime.datetime.now(tz).strftime("%x %X"), signal, signals[signal]))
                            api.submit_order(signal, 1, 'buy', 'market', 'day')
                            # print(datetime.datetime.now(tz).strftime("%x %X"), 'buying', signals[signal], signal)
                    else:
                        try:
                            api.submit_order(signal, 1, 'sell', 'market', 'day')
                            logging.warning('{} {} - {}'.format(datetime.datetime.now(tz).strftime("%x %X"), signal, signals[signal]))
                        except Exception as e:
                            # print('No sell', signal, e)
                            pass

                time.sleep(60)
            else:
                # Get time amount until open, sleep that amount
                print('Market closed ({})'.format(datetime.datetime.now(tz)))
                print('Sleeping', round(time_to_open(datetime.datetime.now(tz))/60/60, 2), 'hours')
                time.sleep(time_to_open(datetime.datetime.now(tz)))
        else:
            # If not trading day, find out how much until open, sleep that amount
            print('Market closed ({})'.format(datetime.datetime.now(tz)))
            print('Sleeping', round(time_to_open(datetime.datetime.now(tz))/60/60, 2), 'hours')
            time.sleep(time_to_open(datetime.datetime.now(tz)))

stocks = ['AA','AAL','AAPL','AIG','AMAT','AMC','AMD','AMGN','AMZN','APA','BA','BABA','BAC','BBY','BIDU','BP','C','CAT','CMG',]

run_checker(stocks)

run_checker started
Market closed (2019-08-17 16:19:46.228207-05:00)
Sleeping 41.17 hours


In [12]:
import alpaca_trade_api as tradeapi

In [1]:
import alpaca_trade_api as tradeapi
# Replace with your keys below
APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'
APCA_API_KEY_ID = 'YOUR_KEY'
APCA_API_SECRET_KEY = 'YOUR_SECRET'

api = tradeapi.REST(
    base_url=APCA_API_BASE_URL,
    key_id=APCA_API_KEY_ID,
    secret_key=APCA_API_SECRET_KEY
)

# Get our account information.
account = api.get_account()

# Check if our account is restricted from trading.
if account.trading_blocked:
    print('Account is currently restricted from trading.')

# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))

$400000 is available as buying power.


In [3]:
# Check if AAPL is tradable on the Alpaca platform.

asset = "AAPL"
try:
    aapl_asset = api.get_asset(asset)
    if aapl_asset.tradable:
        print("We can trade " + asset)
    else:
        print("We cannot trade " + asset)
except:
    print("Error finding " + asset)

We can trade AAPL


In [5]:
# Get a list of all active assets.
active_assets = api.list_assets(status='active')

# Filter the assets down to just those on NASDAQ.
nasdaq_assets = [a for a in active_assets if a.exchange == 'NASDAQ']
for i in nasdaq_assets:
    print(i)

Asset({   'class': 'us_equity',
    'easy_to_borrow': False,
    'exchange': 'NASDAQ',
    'id': '9a22ff6e-bbf1-4f57-9d4f-9a62aa03a5e2',
    'marginable': False,
    'shortable': False,
    'status': 'active',
    'symbol': 'IGLD',
    'tradable': True})
Asset({   'class': 'us_equity',
    'easy_to_borrow': False,
    'exchange': 'NASDAQ',
    'id': 'f757e147-82de-4aac-af47-f1c69f648701',
    'marginable': True,
    'shortable': False,
    'status': 'active',
    'symbol': 'PLIN',
    'tradable': True})
Asset({   'class': 'us_equity',
    'easy_to_borrow': False,
    'exchange': 'NASDAQ',
    'id': 'd945d683-138d-4cbe-8ca0-cf279305e93a',
    'marginable': True,
    'shortable': False,
    'status': 'active',
    'symbol': 'TWMC',
    'tradable': True})
Asset({   'class': 'us_equity',
    'easy_to_borrow': False,
    'exchange': 'NASDAQ',
    'id': 'f5f6b628-333c-4980-aadd-8b39afa53f09',
    'marginable': True,
    'shortable': False,
    'status': 'active',
    'symbol': 'HONE',
    't

In [6]:
# Check if the market is open now.
clock = api.get_clock()
print('The market is {}'.format('open.' if clock.is_open else 'closed.'))

# Check when the market was open on Dec. 1, 2018
date = '2019-08-17'
calendar = api.get_calendar(start=date, end=date)[0]
print('The market opened at {} and closed at {} on {}.'.format(
    calendar.open,
    calendar.close,
    date
))

The market is closed.
The market opened at 09:30:00 and closed at 16:00:00 on 2019-08-17.


In [7]:
# Get daily price data for AAPL over the last 5 trading days.
barset = api.get_barset('AAPL', 'day', limit=5)
aapl_bars = barset['AAPL']

# See how much AAPL moved in that timeframe.
week_open = aapl_bars[0].o
week_close = aapl_bars[-1].c
percent_change = (week_close - week_open) / week_open
print('AAPL moved {}% over the last 5 days'.format(percent_change))

AAPL moved 0.0341649133353371% over the last 5 days


In [8]:
# Submit a market order and assign it a Client Order ID.
###
api.submit_order(
    symbol='AAPL',
    qty=1,
    side='buy',
    type='market',
    time_in_force='gtc',
    client_order_id='my_first_order'
)

# Get our order using its Client Order ID.
my_order = api.get_order_by_client_order_id('my_first_order')
print('Got order #{}'.format(my_order.id))

Got order #38673b5f-2150-4505-8bb4-2e5703501e5f


In [9]:
# Get the last 100 of our closed orders
closed_orders = api.list_orders(
    status='closed',
    limit=100
)

# Get only the closed orders for a particular stock
closed_aapl_orders = [o for o in closed_orders if o.symbol == 'AAPL']
print(closed_aapl_orders)

[]


In [15]:
# Get our position in AAPL.
aapl_position = api.get_position('AAPL')

# Get a list of all of our positions.
portfolio = api.list_positions()

# Print the quantity of shares for each position.
for position in portfolio:
    print("{} shares of {}".format(position.qty, position.symbol))

APIError: position does not exist

In [23]:
!pip install QuantLib-Python
!pip install scipy
!pip install zipline

     |████████████████████████████████| 2.5MB 6.8MB/s eta 0:00:01
     |████████████████████████████████| 92kB 29.7MB/s eta 0:00:01
     |████████████████████████████████| 26.3MB 42.5MB/s eta 0:00:01
     |████████████████████████████████| 122kB 51.4MB/s eta 0:00:01
     |████████████████████████████████| 143kB 54.1MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 47.2MB/s eta 0:00:01
     |████████████████████████████████| 624kB 49.7MB/s eta 0:00:01
     |████████████████████████████████| 471kB 48.7MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 47.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 22.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 29.5MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/a4/d6/67/f303ab028b004bf8e00c05b5b04fba83d8ec238b6547becdb7
  Stored in directory: /home/dsxuser/.cache/pip/wheels/a2/9f/6f/8c7a4ed6b9f6f3c98b742dbb0fd41fff3c130119c507376301
  Stored in directory: /home/dsxuser

In [28]:
# Research environment functions
from zipline.api import prices, symbol

# Pandas library: https://pandas.pydata.org/
import pandas as pd

# Query historical pricing data for AAPL
aapl_close = prices(
    assets=symbol('AAPL'),
    start='2013-01-01',
    end='2016-01-01',
)

# Compute 20 and 50 day moving averages on
# AAPL's pricing data
aapl_sma20 = aapl_close.rolling(20).mean()
aapl_sma50 = aapl_close.rolling(50).mean()

# Combine results into a pandas DataFrame and plot
pd.DataFrame({   
    'AAPL': aapl_close,
    'SMA20': aapl_sma20,
    'SMA50': aapl_sma50
}).plot(
    title='AAPL Close Price / SMA Crossover'
);

ValueError: index must be monotonic increasing or decreasing

In [19]:
 import alpaca_trade_api as tradeapi

api = tradeapi.REST('YOUR_KEY',
                   'YOUR_SECRET')

# Get daily price data for AAPL over the last 5 trading days.
barset = api.get_barset('AAPL', 'day', limit=5)
aapl_bars = barset['AAPL']

# See how much AAPL moved in that timeframe.
week_open = aapl_bars[0].o
week_close = aapl_bars[-1].c
percent_change = (week_close - week_open) / week_open * 100
print('AAPL moved {}% over the last 5 days'.format(percent_change)) 

AAPL moved 3.41649133353371% over the last 5 days


In [45]:
 import alpaca_trade_api as tradeapi
import time
import datetime
from datetime import timedelta
from pytz import timezone
import logging

tz = timezone('EST')

api = tradeapi.REST('YOUR_KEY',
                   'YOUR_SECRET')


# Mean Reversion
def get_pricedata(myList):
    for i in range(len(mylist)):
        barset = api.get_barset(mylist[i], 'day', limit=5)
        aapl_bars = barset[mylist[i]]
        # See how much AAPL moved in that timeframe.
        week_open = aapl_bars[0].o
        week_close = aapl_bars[-1].c
        percent_change = (week_close - week_open) / week_open * 100
        return "AAPL moved {}% over the last 5 days".format(percent_change)
    
        logging.basicConfig(filename='./new_5min_ema.log', format='%(name)s - %(levelname)s - %(message)s')
        logging.warning('{} logging started'.format(datetime.datetime.now().strftime("%x %X")))

def get_data_bars(symbols, rate, slow, fast):
   data = api.get_barset(symbols, rate, limit=20).df
   for x in symbols:
       data.loc[:, (x, 'fast_ema')] = data[x]['close'].rolling(window=fast).mean()
       data.loc[:, (x, 'slow_ema')] = data[x]['close'].rolling(window=slow).mean()
   return data

def get_signal_bars(symbol_list, rate, ema_slow, ema_fast):
   data = get_data_bars(symbol_list, rate, ema_slow, ema_fast)
   signals = {}
   for x in symbol_list:
       if data[x].iloc[-1]['fast_ema'] > data[x].iloc[-1]['slow_ema']: signal = 1
       else: signal = 0
       signals[x] = signal
   return signals

def time_to_open(current_time):
   if current_time.weekday() <= 4:
       d = (current_time + timedelta(days=1)).date()
   else:
       days_to_mon = 0 - current_time.weekday() + 7
       d = (current_time + timedelta(days=days_to_mon)).date()
   next_day = datetime.datetime.combine(d, datetime.time(9, 30, tzinfo=tz))
   seconds = (next_day - current_time).total_seconds()
   return seconds

def run_checker(stocklist):
   print('run_checker started')
   change = get_pricedata(stocklist)
   print(change)
   

stocks = ['AA','AAL','AAPL','AIG','AMAT','AMC','AMD','AMGN','AMZN','APA','BA','BABA','BAC','BBY','BIDU','BP','C','CAT','CMG',]

run_checker(stocks)

SyntaxError: invalid syntax (<ipython-input-45-414609d57aa6>, line 15)